# NOTEBOOK PARA GENERAR EL DATAFRAME CON LA DATA DE TODOS LOS ARCHIVOS PARA AAFIT, BBFIT Y OTROS

<font size=5>La data para los hits la generaré con otro Notebook.</font>

Es importante mencionar que esta "limpieza" (generación del algoritmo) y obtención de data, me llevó aproximadamente 12 horas en general, desde el proceso creativo, pasando por la frustración, muchos intentos fallidos y hasta el momento de EUREKA!!!... XD.

Borré ya la mayoría de celdas que me llevaron al código de las siguientes dos celdas, el cual está mucho más pulido y limpio, pero creo que es importente mencionar por lo que pase y las ideas (algunas locas) que me surgieron para poder resolverlo.

Algunos ideas que tuve antes de llegar a decidir que no calcularia como tal el indice de aafit fueron:
* Solo quedarme con los archivos que tuvieran 100% data completa (terrible idea ya que son más del 50% los archivos que en algunos eventos - alrededor del 5% o menos - no tienen data de aafit).
* Verificar las longitudes de la cadena aafit y bbfit y comparar (idea bastante mala por cierto)
* Tratar de buscar por separado en otra celda solo la data de aafit y "pegarla" de alguna manera al dataframe final, hay que decir que esta idea también fué muy mala y no prosperó en lo absoluto (creo no tenia ni pies ni cabeza, momento de frustración máxima).

**NOTA:** Como siempre, la gran cuestión y prerrogativa humana es encontrar patrones en los datos y automatizar
búsquedas aún y cuando estos no se vean a simple vista o no sean fácilmente inferibles, tales como cuando 
el patrón es precisamente eso... que NO HAY PATRÓN!!!, esto aplica para el no indice ind_aafit

In [1]:
#Recordando algunas variables importantes, funciones y librerías que se requerirán
import os
import pandas as pd
import re
import math

#extractpath = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este path es para mi laptop en casa
extractpath = "F:\\DATA_TFM\\TXTs" #Este path es para mi PC en el trabajo
#extractpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este es para mi no tan nueva PC-Desktop

txtfilesarr = os.listdir(extractpath); txtfilesarr=sorted(txtfilesarr)
csv_path = "F:\\DATA_TFM\\CSVs\\"
#csv_path = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\"
csvarr = os.listdir(csv_path);csvarr=sorted(csvarr)

# Funcion para obtener los indices de un archivo
def indices(df):
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = [];#ind_aafit = []
    for i in range(data.shape[0]):
        tempstr = data[0][i]
        if "start_event" in tempstr: ind_startev.append(i)
        if "UTC" in tempstr: ind_runid.append(i)
        if "weights" in tempstr: ind_weights.append(i)
        if "muon" in tempstr: ind_muon.append(i)
        #if "aafit" in tempstr:  ##NO LO CALCULARE!!! OBTENDRE LOS VALORES DE OTRA FORMA
        #    ind_aafit.append(i) ##COMO TENGO EL INDICE DE BBFIT, RESTO 1 LINEA Y ASIGNO VALORES
        if "bbfit" in tempstr:   ##SEGUN LA EXISTENCIA O NO DE LA LINEA!!!! EUREKA!!!
            ind_bbfit.append(i)
        if "selected" in tempstr: ind_bb_selp.append(i)
        if "end_event" in tempstr: ind_endev.append(i)
    return(ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev)

#Función para verificar si los indices tienen el mismo # de elementos
def longitudes(i1,i2,i3,i4,i5,i6,i7):
    a=len(i1);b=len(i2);c=len(i3);d=len(i4);e=len(i5);f=len(i6);g=len(i7)
    #print(a,b,c,d,e,f,g)
    if (a==b and b==c and c==d and d==e and e==f and f==g): return(True)
    else: return(False)
    return()

### NOTA: Como siempre, la gran cuestión y prerrogativa humana es encontrar patrones en los datos y automatizar
### búsquedas aún y cuando estos no se vean a simple vista o no sean fácilmente inferibles, tales como cuando 
### el patrón es precisamente eso... que NO HAY PATRÓN!!!, esto aplica para el no indice ind_aafit

In [4]:
############################################################################################################
###### ESTE CODIGO GENERARÁ UN DATAFRAME CON LA DATA DE BBFIT, AAFIT Y OTRAS DE ""TODOS"" LOS ARCHIVOS #####
############################################################################################################

#Nota: Esta nueva extracción comenzó el 8 Jul a las 17:20

#Se requiere un dataframe donde iremos guardando cada fila de data obtenida de cada archivo
columnas = ['runID','frameID','trigger_counter','interactionID','aafit_azimut',
            'aafit_zenit','aafit_lambda','aafit_beta','bbfit_azimut','bbfit_zenit','bbfit_quality',
            'muon_azimut','muon_zenit','muon_energy'] #La data del muon la añadi el 30 Jun, lo habia "olvidado" XD
#df_data1 = pd.DataFrame(columns=columnas)

datos = []

#consecutivo=0 #variable auxiliar para guardar el indice del ultimo elemento del archivo

# Bucle para recorrer TODOS los archivos y obtener la data y metadata de TODOS
for k in range(len(csvarr)):
    #print("Ciclo", k)
    file = csv_path+csvarr[k]
    print("El archivo trabajando es: ",file)
    data = pd.read_csv(file, header=None) # Dataframe del archivo en tratamiento
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = []

    #Generacion de indices para cada archivo
    ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)

    #Como el runid es identico para todos los datos en un mismo archivo, uso el primero que aparece en el dataframe
    runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])
    #Igual como el interactionID es el mismo para todos, asigno 1 si el archivo es "numu" y 2 si es "anumu"
    interactionID = [lambda:1, lambda:2]["anumu" in str(file)]()
    # Bucle para recorrer el archivo en curso y obtener la data relevante
    print("Entrando al bucle para meter renglones al dataframe") 
    for i in range(len(ind_startev)):
        #Codigo para obtener los demas datos por cada evento y despues asignarlo a los hits
        #Divido la cadena en los valores que la componen para después extraer frameid y trigger_counter
        temprunid = list(data[0][ind_runid[i]].split(" "))
        tempmuon = list(data[0][ind_muon[i]].split(" "))
        tempbbfit = list(data[0][ind_bbfit[i]].split(" "))
        #Extrayendo las variables frameid, trigger_counter de este renglon---> esta data si cambia ojo!!
        frameID = temprunid[1]; trigger_counter = temprunid [2]
        #Obteniendo y generando Datos para aafit
        #Para calcular la data solo haremos la diferencia y si exista data la calculamos si no
        #pues ponemos valores fuera de rango para saber después que no hubo data como -3pi
        laafit = data[0][ind_bbfit[i]-1]
        if "aafit" in laafit:
            tempaafit = list(data[0][ind_bbfit[i]-1].split(" "))
            aafit_azimut = math.atan2(float(tempaafit[3]),float(tempaafit[2]))
            aafit_zenit = math.acos(float(tempaafit[4]))
            aafit_lambda = tempaafit[-2]
            aafit_beta = tempaafit [-1]
        else:
            aafit_azimut=-3*math.pi;aafit_zenit = -3*math.pi;aafit_lambda=1;aafit_beta=-1
        #Obteniendo y generando Datos para bbfit
        X = float(tempbbfit[2]); Y = float(tempbbfit[3]); Z = float(tempbbfit[4])
        #Hay que verificar si X o Y son nan y asignar el valor de bbfit_azimut
        if math.isnan(X):   bbfit_azimut = -3*math.pi
        elif math.isnan(Y): bbfit_azimut = -3*math.pi
        else: bbfit_azimut = math.atan2(Y,X)
        bbfit_zenit = math.acos(Z)
        bbfit_quality = float(tempbbfit[-1]) 
        #Obteniendo y generando la data del muon
        X = float(tempmuon[1]); Y = float(tempmuon[2]); Z = float(tempmuon[3])
        muon_azimut = math.atan2(Y,X)
        muon_zenit = math.acos(Z)
        muon_energy = float(tempmuon[-2])
        #Añadimos la data obtenida y la generada al dataframe
        renglon = [runID, frameID, trigger_counter, interactionID,aafit_azimut, aafit_zenit,
                   aafit_lambda,aafit_beta, bbfit_azimut,bbfit_zenit,bbfit_quality,muon_azimut,muon_zenit,muon_energy]
        #Añadiendo datos a una lista para posteriormente convertirlo en dataframe.
        datos.append(renglon)     

print("Fin de la extracción de la data")

print("Creando el dataframe")

#Añadiendo datos al dataframe.
df_data1 = pd.DataFrame(datos, columns=columnas)

El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_025800_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_025800_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_025880_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_025880_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_025920_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_025920_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_025930_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_025930_numu_CC_a_reco.i3.csv
Entrando al bucle para m

Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_027130_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_027170_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_027170_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_027180_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_027180_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_027230_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_027230_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_02

Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_028630_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_028650_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_028650_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_028720_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_028720_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_028730_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_028730_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_02

Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_029420_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_029440_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_029440_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_029480_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_029480_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_029490_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_029490_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_02

Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_030220_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_030240_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_030240_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_030310_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_030310_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_030320_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_030320_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_03

...

Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_083780_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_083780_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_083800_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_083800_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_083810_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_083810_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_083820_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_0

Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084210_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084210_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084230_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084230_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084240_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084240_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084250_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_0

Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084590_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084590_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084600_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084600_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084610_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084610_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084620_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_0

Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084980_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084980_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084990_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_084990_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_085000_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_085000_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_085010_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_0

Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_085410_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_085430_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_085430_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_085440_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_085440_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_085450_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_085450_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_08

In [6]:
### PRUEBAS... BORRAR ESTA CELDA AL FINAL

#Viendo como quedó el dataframe
print("El número de filas del dataframe es:", df_data1.shape[0])

print("Una muestra de las filas y la data en ellas")

df_data1.sample(10)

El número de filas del dataframe es: 2261213
Una muestra de las filas y la data en ellas


,runID,frameID,trigger_counter,interactionID,aafit_azimut,aafit_zenit,aafit_lambda,aafit_beta,bbfit_azimut,bbfit_zenit,bbfit_quality,muon_azimut,muon_zenit,muon_energy
1764587,75950,22215,1057,2,-2.841978,0.646447,-6.07766209382,0.0131825178004,-9.424778,0.530765,1.000061,-2.884103,0.631488,85.5279
1885869,81200,360751,678,2,-1.021116,0.712738,-5.95753325088,0.12968730783,-9.424778,0.991796,2.569131,-0.470119,0.923051,11.7549
730536,52610,80894,1384,2,-1.716690,0.992192,-5.01373447221,0.00949840148736,-2.207173,0.937809,1.074719,-1.722539,0.975174,592.2750
1278768,62440,60229,1027,2,-2.244093,1.442742,-5.92952832136,0.00799076362344,-2.148944,1.434144,1.195356,-2.148200,1.437681,351.8810
713094,52340,2568,2809,2,-1.549409,1.247397,-5.27281397886,0.0060940778734,-1.518924,1.267584,2.748742,-1.553510,1.243934,145.3860
886381,55020,18550,1595,1,0.905484,0.627799,-6.057802432,0.0180370448141,-9.424778,0.509948,1.420427,-1.507526,0.651470,32.6524
1915103,82270,100929,975,2,-1.719807,0.223578,-5.35370267093,0.00659044174053,-9.424778,0.208101,0.479564,-1.667396,0.261949,123.0610
279616,37280,40809,905,2,-1.897461,0.534172,-5.7138775235,0.00671459494405,-9.424778,0.392660,0.657958,-1.919864,0.530275,70.9834
1236894,61980,28654,2307,2,1.776230,1.477825,-5.89844747241,0.00784403349893,-9.424778,1.434609,2.061933,1.626877,1.416956,73.9961
2048909,83660,28420,824,1,-3.024619,0.845082,-5.80142943961,0.0353824017906,-9.424778,1.033633,2.185441,3.011624,1.162085,45.7966


In [7]:
## Una vez que tenemos el dataframe con toda la data relevante de aafit y bbfit
## debemos guardar esta data a disco

#outputpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CLEANDATA\\"
outputpath = "F:\\DATA_TFM\\CLEANDATA\\"
    
df_data1.to_csv (outputpath+"data_aafit_bbfit_muon.csv", index = None)#, header=None)

# EL SIGUIENTE CÓDIGO NO ES NECESARIO PERO LO DEJÉ COMO EVIDENCIA DE LO QUE HABÍA ESTADO HACIENDO INICIALMENTE

In [ ]:
##########################################################################################
####  ESTE CODIGO CONTIENE EL PRIMER ALGORITMO QUE GENERÉ PARA OBTENER LA DATA DE      ###
####  UN ARCHIVO CSV YA CON TRATAMIENTO "INICIAL", ES DECIR SIN LINEAS VACIAS Y OTRAS  ### 
####          ESTABA DIVIDIDO EN DOS CELDAS POR TODO EL TESTING QUE HICE               ###
##########################################################################################

    ####  #####  #####  #####  #####
    ##    ####   ####   #   #  ####
    ####  #   #  #   #  #####  #   #
    
# -->ESTE ALGORITMO TUVO UN FALLO FUNDAMENTAL... NO TOMABA EN CUENTA QUE HAY ARCHIVOS QUE NO TIENEN DATA DE AAFIT
# -->Sin embargo me sirvió como base primaria para generar el adecuado (2da. celda de arriba a abajo de este notebook)

file = csv_path+csvarr[1]
print("El archivo trabajando es: ",file)
data = pd.read_csv(file, header=None) # Dataframe del archivo en tratamiento
ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
ind_aafit = [];ind_bbfit = [];ind_bb_selp = [];ind_endev = []

for i in range(data.shape[0]):
    tempstr = data[0][i]
    if "start_event" in tempstr:
        ind_startev.append(i)
    if "UTC" in tempstr:
        ind_runid.append(i)
    if "weights" in tempstr:
        ind_weights.append(i)
    if "muon" in tempstr:
        ind_muon.append(i)
    if "aafit" in tempstr:
        ind_aafit.append(i)
    if "bbfit" in tempstr:
        ind_bbfit.append(i)
    if "selected" in tempstr:
        ind_bb_selp.append(i)
    if "end_event" in tempstr:
        ind_endev.append(i)

####### YA QUE LOGRE GENERAR EL DATAFRAME PARA LOS DATOS DE AAFIT Y BBBFIT DE UN SOLO EVENTO,
####### AHORA LO HARÉ PARA TODOS LOS EVENTOS EN EL MISMO ARCHIVO
####### CODIGO PARA OBTENER LA DATA AAFIT, BBFIT, TRANSFORMACIONES Y OTROS PARA EL DATAFRAME

#Como el runid es identico para todos los datos en un mismo archivo, uso el primero que aparece en el dataframe
runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])

#Igual como el interactionID es el mismo para todos, asigno 1 si el archivo es "numu" y 2 si es "anumu"
if "anumu" in str(file):
    interactionID = 2
else:
    interactionID = 1
    
#Se requiere un dataframe donde iremos guardando cada fila de data obtenida
columnas = ['runID','frameID','trigger_counter','interactionID','aafit_azimut','aafit_zenit',
            'aafit_lambda','aafit_beta','bbfit_azimut','bbfit_zenit','bbfit_quality']
df_data1 = pd.DataFrame(columns=columnas)

#Ahora generamos un bucle para recorrer todo el archivo y usamos los indices
for i in range(len(ind_startev)):
    #Codigo para obtener los demas datos por cada evento y despues asignarlo a los hits
    #Divido la cadena en los valores que la componen para después extraer frameid y trigger_counter
    temprunid = list(data[0][ind_runid[i]].split(" "))
    tempmuon = list(data[0][ind_muon[i]].split(" "))
    tempaafit = list(data[0][ind_aafit[i]].split(" "))
    tempbbfit = list(data[0][ind_bbfit[i]].split(" "))
    #Extrayendo las variables frameid, trigger_counter de este renglon---> esta data si cambia ojo!!
    frameID = temprunid[1]; trigger_counter = temprunid [2]
    #Obteniendo y generando Datos para aafit
    aafit_azimut = math.atan2(float(tempaafit[3]),float(tempaafit[2]))
    aafit_zenit = math.acos(float(tempaafit[4]))
    aafit_lambda = tempaafit[-2]
    aafit_beta = tempaafit [-1]
    #Obteniendo y generando Datos para bbfit
    X = float(tempbbfit[2]); Y = float(tempbbfit[3]); Z = float(tempbbfit[4])
    #Hay que verificar si X o Y son nan y asignar el valor de bbfit_azimut
    if math.isnan(X):   bbfit_azimut = -3*math.pi
    elif math.isnan(Y): bbfit_azimut = -3*math.pi
    else: bbfit_azimut = math.atan2(Y,X)
    bbfit_zenit = math.acos(Z)
    bbfit_quality = float(tempbbfit[-1]) 
    #Añadimos la data obtenida y la generada al dataframe
    renglon = [runID, frameID, trigger_counter, interactionID,aafit_azimut, aafit_zenit,
               aafit_lambda,aafit_beta, bbfit_azimut,bbfit_zenit,bbfit_quality]
    df_data1.loc[i] = renglon
    
